---
title: Python 多进程 Multiprocessing
tags: 小书匠,Multiprocessing,Python,apply,map,Pool
grammar_cjkRuby: true
renderNumberedHeading: true
---

[toc!]

# Python 多进程 Multiprocessing

- 多进程的使用（适用于CPU密集型操作）
- 使用`multiprocessing`.

## 简单使用

### 实现方法一

In [7]:
from multiprocessing import  Process

def fun1(name):
    print('测试%s多进程' %name)

process_list = []
for i in range(5):  #开启5个子进程执行fun1函数
    p = Process(target=fun1, args=('Python',)) #实例化进程对象
    p.start()
    process_list.append(p)

for i in process_list:
    p.join()

print('结束测试')

测试Python多进程
测试Python多进程
测试Python多进程
测试Python多进程
测试Python多进程
结束测试


### 实现方法二

In [8]:
from multiprocessing import  Process

class MyProcess(Process): #继承Process类
    def __init__(self,name):
        super(MyProcess,self).__init__()
        self.name = name

    def run(self):
        print('测试%s多进程' % self.name)


process_list = []
for i in range(5):  #开启5个子进程执行fun1函数
    p = MyProcess('Python') #实例化进程对象
    p.start()
    process_list.append(p)

for i in process_list:
    p.join()

print('结束测试')

测试Python多进程
测试Python多进程
测试Python多进程
测试Python多进程
测试Python多进程
结束测试


## 使用进行池

这是用生产消费者模式来进行任务调度，是比较常规的使用场景，Pool() 内部维护了一个任务队列，类似于生产者消费之模型

### 使用 apply 函数

### func 是单参数函数的情形

In [28]:
from multiprocessing import Pool

def func(i):
    print('Worker %s starts!'%i)
    return i

results = []
pool = Pool()
for i in range(5):
    # 用error_callback 来处理使用子进程的错误信息，否则子进程如果报错也不会出现错误提示
    result = pool.apply_async(func, args=(i,), error_callback=print)
    results.append(result)
pool.close()
pool.join()

# 如果函数有返回，可以通过.get()方法获得
for result in results:
    print(result.get())

Worker 0 starts!
Worker 1 starts!
Worker 2 starts!
Worker 3 starts!
Worker 4 starts!
0
1
2
3
4


### func 是多参数函数的情形

In [34]:
from multiprocessing import Pool

def func(i, j):
    print('Worker %s starts! I print %d' %(i, j))
    return i, j

results = []
pool = Pool()
for i in range(5):
    # 用error_callback 来处理使用子进程的错误信息，否则子进程如果报错也不会出现错误提示
    result = pool.apply_async(func, args=(i, i+1), error_callback=print)
    results.append(result)
pool.close()
pool.join()

# 如果函数有返回，可以通过.get()方法获得
for result in results:
    print(result.get())

Worker 3 starts! I print 4
Worker 1 starts! I print 2
Worker 2 starts! I print 3
Worker 0 starts! I print 1
Worker 4 starts! I print 5
(0, 1)
(1, 2)
(2, 3)
(3, 4)
(4, 5)


注意：
1. 注意，如果func不接受参数，要写成 args=(); 如果 func 只接受一个参数i，一定要写成(i,)，不能写成(i)，因为 (i, ) 是元素为 i 的 tuple，而 (i) 相当于 i，不是 tuple; 如果用 list 传参就没有这个问题，直接用 [i] 就可以。
2. 使用 error_callback函数来指定报错
3. Pool() 没有参数时，默认创建 `os.cpu_count()` 多个线程

### 使用 map 类型

#### map

pool.map 类似与内置的 map 函数，只能对**单参数函数**使用

In [29]:
def func(x):
    print(x)
    return x

pool = Pool()
ret = pool.map(func, range(5)) #关键点，images是一个可迭代对象
pool.close()
pool.join()
print(ret) # 结果直接可以用，不像 apply 的结果，还需要 .get() 获取

0
3
1
2
4
[0, 1, 2, 3, 4]


#### starmap

starmap 可以对 **多参数函数** 使用，但是**不能**对单参数函数使用。

In [36]:
def func(x, y):
    print(x + y)

pool = Pool()
pool.starmap(func, zip(range(5), range(5))) #关键点，images是一个可迭代对象
pool.close()
pool.join()

6
0
4
2
8



## 其它辅助

### 获取 cpu 数

In [18]:
import os
os.cpu_count()

8

# References
1. http://localhost:8888/lab/workspaces/auto-K/tree/learnPython/Python%20%E5%A4%9A%E8%BF%9B%E7%A8%8B%20multiprocess.ipynb
5. [multiprocessing — Process-based parallelism — Python 3.9.2 documentation](https://docs.python.org/3/library/multiprocessing.html#multiprocessing.pool.Pool.starmap)
3. [python - How to use multiprocessing pool.map with multiple arguments? - Stack Overflow](https://stackoverflow.com/questions/5442910/how-to-use-multiprocessing-pool-map-with-multiple-arguments)
1. [Python程序中的进程操作-进程池(multiprocess.Pool） - 二十三岁的有德 - 博客园](https://www.cnblogs.com/nickchen121/p/11130258.html#%E5%85%AB%E3%80%81%E5%9B%9E%E8%B0%83%E5%87%BD%E6%95%B0)
2. [一篇文章搞定Python多进程(全) - 知乎](https://zhuanlan.zhihu.com/p/64702600)